# 04 - Grok Integration

## Agentic Logistics Control System

This notebook implements integration with xAI's Grok LLM:
- API client setup and configuration
- Rate limiting and retry logic
- Error handling and fallbacks
- Response parsing and validation

### Control Loop Role: Foundation for REASONING

```
OBSERVE -> [REASON] -> PLAN -> DECIDE -> ACT -> FEEDBACK -> (loop)
            ^
            |
      Grok powers this
```

In [ ]:
# Setup
import sys
from pathlib import Path

PROJECT_ROOT = Path.cwd().parent
sys.path.insert(0, str(PROJECT_ROOT))
sys.path.insert(0, str(PROJECT_ROOT / "src"))

In [ ]:
import os
from config.settings import settings

print("Current configuration:")
print(f"  API Base URL: {settings.XAI_BASE_URL}")
print(f"  Model: {settings.GROK_MODEL}")
print(f"  Temperature: {settings.LLM_TEMPERATURE}")
print(f"  Max Tokens: {settings.LLM_MAX_TOKENS}")
print(f"  API Key configured: {bool(settings.XAI_API_KEY)}")

## 1. Grok Client Setup

In [ ]:
from src.reasoning.grok_client import GrokClient, get_grok_client

# Create client
client = GrokClient()

print(f"Client Status:")
print(f"  Is Available: {client.is_available}")
print(f"  Model: {client.model}")
print(f"  Temperature: {client.temperature}")
print(f"  Max Tokens: {client.max_tokens}")

In [ ]:
# Test basic completion (will use fallback if API key not configured)
response = client.complete(
    prompt="What are the top 3 factors to consider when optimizing truck routes?",
    system_prompt="You are a logistics expert. Be concise."
)

print(f"Response:")
print(f"  Success: {response['success']}")
print(f"  Model: {response['model']}")
print(f"  Finish Reason: {response['finish_reason']}")
print(f"\nContent:\n{response['content'][:500]}..." if len(response['content']) > 500 else f"\nContent:\n{response['content']}")

## 2. JSON Mode Completion

In [ ]:
# Test JSON completion
json_response = client.complete_json(
    prompt="""Analyze this truck status:
    - Truck ID: TRK-001
    - Speed: 5 km/h for last 15 minutes
    - Expected speed: 60 km/h
    - Traffic: Heavy on current segment
    
    Return a JSON object with: is_stuck (bool), severity (low/medium/high), recommendation (string)""",
    system_prompt="You are a logistics analyst. Always respond with valid JSON."
)

print(f"JSON Response:")
print(f"  Success: {json_response['success']}")
print(f"  Parsed: {json_response.get('parsed')}")

if json_response.get('parse_error'):
    print(f"  Parse Error: {json_response['parse_error']}")

## 3. Error Handling and Fallbacks

In [ ]:
# Test with intentionally unavailable client
test_client = GrokClient(api_key="")  # Empty API key

print(f"Test client available: {test_client.is_available}")

# Should trigger fallback
fallback_response = test_client.complete(
    prompt="Test prompt for issue detection",
    system_prompt="Test system"
)

print(f"\nFallback Response:")
print(f"  Success: {fallback_response['success']}")
print(f"  Model: {fallback_response['model']}")
print(f"  Error: {fallback_response.get('error')}")
print(f"  Content: {fallback_response['content'][:200]}...")

## 4. Usage Statistics

In [ ]:
# Check client statistics
stats = client.get_stats()

print("Client Statistics:")
for key, value in stats.items():
    print(f"  {key}: {value}")

## 5. Logistics-Specific Testing

In [ ]:
# Test logistics-specific prompt
logistics_prompt = """
Analyze the following fleet situation:

Fleet Status:
- 5 trucks active
- 2 trucks en-route, both reporting slow speeds (15 km/h)
- 1 truck idle at warehouse
- 2 trucks loading/unloading

Traffic:
- I-95 Northbound: Heavy traffic, 25 km/h average
- Route 9A: Light traffic, 55 km/h average

Pending Deliveries:
- 3 deliveries due within 2 hours (HIGH priority)
- 2 deliveries due within 4 hours (NORMAL priority)

Provide analysis in JSON format with: situation_summary, issues (array), recommendations (array)
"""

logistics_response = client.complete_json(
    prompt=logistics_prompt,
    system_prompt="You are an expert logistics fleet manager. Analyze situations and provide actionable insights."
)

print("Logistics Analysis:")
if logistics_response.get('parsed'):
    import json
    print(json.dumps(logistics_response['parsed'], indent=2))
else:
    print(logistics_response['content'])

## 6. Singleton Pattern

In [ ]:
# Test singleton access
client1 = get_grok_client()
client2 = get_grok_client()

print(f"Singleton test: {client1 is client2}")
print(f"Client available: {client1.is_available}")

## 7. Integration Test

In [ ]:
print("=" * 50)
print("GROK INTEGRATION TEST")
print("=" * 50)

tests_passed = 0
tests_total = 0

# Test 1: Client creation
tests_total += 1
try:
    test_client = GrokClient()
    assert test_client is not None
    print("✓ Test 1: Client creation")
    tests_passed += 1
except Exception as e:
    print(f"✗ Test 1: {e}")

# Test 2: Complete method
tests_total += 1
try:
    response = test_client.complete("Hello")
    assert 'content' in response
    assert 'success' in response
    print("✓ Test 2: Complete method")
    tests_passed += 1
except Exception as e:
    print(f"✗ Test 2: {e}")

# Test 3: JSON complete method
tests_total += 1
try:
    response = test_client.complete_json("Return {\"test\": true}")
    assert 'content' in response
    print("✓ Test 3: JSON complete method")
    tests_passed += 1
except Exception as e:
    print(f"✗ Test 3: {e}")

# Test 4: Fallback handling
tests_total += 1
try:
    fallback_client = GrokClient(api_key="")
    response = fallback_client.complete("Test")
    assert response['model'] == 'fallback'
    print("✓ Test 4: Fallback handling")
    tests_passed += 1
except Exception as e:
    print(f"✗ Test 4: {e}")

# Test 5: Stats tracking
tests_total += 1
try:
    stats = test_client.get_stats()
    assert 'total_requests' in stats
    assert 'is_available' in stats
    print("✓ Test 5: Stats tracking")
    tests_passed += 1
except Exception as e:
    print(f"✗ Test 5: {e}")

# Test 6: Singleton
tests_total += 1
try:
    c1 = get_grok_client()
    c2 = get_grok_client()
    assert c1 is c2
    print("✓ Test 6: Singleton pattern")
    tests_passed += 1
except Exception as e:
    print(f"✗ Test 6: {e}")

print("=" * 50)
print(f"Tests passed: {tests_passed}/{tests_total}")
if tests_passed == tests_total:
    print("✓ All tests passed!")

## Configuration Notes

To use Grok with a real API key:

1. Copy `.env.template` to `.env`
2. Set `XAI_API_KEY=your-api-key`
3. Restart the notebook

The client will automatically use fallback responses when the API is unavailable.

## Next Steps

1. Proceed to `05_reasoning_prompts.ipynb` for prompt engineering
2. Then `06_reasoning_layer.ipynb` for complete reasoning implementation